<a href="https://colab.research.google.com/github/gnitnaw/LDL/blob/main/pt_framework/c11e1_autocomplete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
The MIT License (MIT)
Copyright (c) 2021 NVIDIA
Permission is hereby granted, free of charge, to any person obtaining a copy of
this software and associated documentation files (the "Software"), to deal in
the Software without restriction, including without limitation the rights to
use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of
the Software, and to permit persons to whom the Software is furnished to do so,
subject to the following conditions:
The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS
FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR
COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER
IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN
CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
"""


'\nThe MIT License (MIT)\nCopyright (c) 2021 NVIDIA\nPermission is hereby granted, free of charge, to any person obtaining a copy of\nthis software and associated documentation files (the "Software"), to deal in\nthe Software without restriction, including without limitation the rights to\nuse, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of\nthe Software, and to permit persons to whom the Software is furnished to do so,\nsubject to the following conditions:\nThe above copyright notice and this permission notice shall be included in all\ncopies or substantial portions of the Software.\nTHE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR\nIMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS\nFOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR\nCOPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER\nIN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OU

This code example demonstrates how to use an LSTM-based neural network and beam search to do text autocompletion. More context for this code example can be found in the section "Programming Example: Using LSTM for Text Autocompletion" in Chapter 11 in the book Learning Deep Learning by Magnus Ekman (ISBN: 9780137470358).


# Google Drive

In [2]:
!wget https://github.com/gnitnaw/LDL/raw/main/pt_framework/utilities.py
from google.colab import drive
drive.mount('/content/drive')
path_head = '/content/drive/MyDrive/Colab Notebooks/' # You have to change this. 

--2023-02-27 20:01:10--  https://github.com/gnitnaw/LDL/raw/main/pt_framework/utilities.py
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/gnitnaw/LDL/main/pt_framework/utilities.py [following]
--2023-02-27 20:01:10--  https://raw.githubusercontent.com/gnitnaw/LDL/main/pt_framework/utilities.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4326 (4.2K) [text/plain]
Saving to: ‘utilities.py’

utilities.py        100%[===================>]   4.22K  --.-KB/s    in 0.09s   

2023-02-27 20:01:11 (45.6 KB/s) - ‘utilities.py’ saved [4326/4326]

Drive already mounted at /content/drive; to attempt to fo

The initialization code is shown in the first code snippet. Apart from the import statements, we need to provide the path to the text file to use for training. We also define two variables, WINDOW_LENGTH and WINDOW_STEP, which are used to control the process of splitting up this text file into multiple training examples. The other three variables control the beam-search algorithm and are described shortly. The text used to train the model is assumed to be in the file ../data/frankenstein.txt.

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
import numpy as np
from utilities import train_model

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
EPOCHS = 32
BATCH_SIZE = 256
# INPUT_FILE_NAME = '../data/frankenstein.txt'
INPUT_FILE_NAME = path_head+'data/frankenstein.txt'
WINDOW_LENGTH = 40
WINDOW_STEP = 3
BEAM_SIZE = 8
NUM_LETTERS = 11
MAX_LENGTH = 50


The next code snippet opens and reads the content of the file, converts it all into lowercase, and replaces double spaces with single spaces. To enable us to easily one-hot encode each character, we want to assign a monotonically increasing index to each character. This is done by first creating a list of unique characters. Once we have that list, we can loop over it and assign an incrementing index to each character. We do this twice to create one dictionary (a hash table) that maps from character to index and a reverse dictionary from index to character. These will come in handy later when we want to convert text into one-hot encoded input to the network as well as when we want to convert one-hot encoded output into characters. Finally, we initialize a variable encoding_width with the count of unique characters, which will be the width of each one-hot encoded vector that represents a character.

In [4]:
# Open the input file.
file = open(INPUT_FILE_NAME, 'r', encoding='utf-8-sig')
text = file.read()
file.close()

# Make lower-case and remove newline and extra spaces.
text = text.lower()
text = text.replace('\n', ' ')
text = text.replace('  ', ' ')

# Encode characters as indices.
unique_chars = list(set(text))
char_to_index = dict((ch, index) for index,
                     ch in enumerate(unique_chars))
index_to_char = dict((index, ch) for index,
                     ch in enumerate(unique_chars))
encoding_width = len(char_to_index)


The next step is to create training examples from the text file. This is done by the next code snippet. Each training example will consist of a sequence of characters and a target output value of a single character immediately following the input characters. We create these input examples using a sliding window of length WINDOW_LENGTH. Once we have created one training example, we slide the window by WINDOW_STEP positions and create the next training example. We add the input examples to one list and the output values to another. All of this is done by the first for loop.

We then create a single array holding all the input examples and another array holding the output values. Both of these arrays will hold data in one-hot encoded form, so each character is represented by a dimension of size encoding_width. We first allocate space for the two arrays and then fill in the values using a nested for loop.


In [5]:
# Create training examples.
fragments = []
targets = []
for i in range(0, len(text) - WINDOW_LENGTH, WINDOW_STEP):
    fragments.append(text[i: i + WINDOW_LENGTH])
    targets.append(text[i + WINDOW_LENGTH])

# Convert to one-hot encoded training data.
X = np.zeros((len(fragments), WINDOW_LENGTH, encoding_width), dtype=np.float32)
y = np.zeros(len(fragments), dtype=np.int64)
for i, fragment in enumerate(fragments):
    for j, char in enumerate(fragment):
        X[i, j, char_to_index[char]] = 1
    target_char = targets[i]
    y[i] = char_to_index[target_char]


We then split the dataset into a training dataset and a test dataset using the train_test_split function from scikit-learn. We only withhold 5% as test dataset. Given that we are mostly interested in inspecting the resulting auto-completions in this example we could have skipped creating a test dataset altogether, but we create it anyway for good practice.

We then convert the arrays into tensors and create Dataset objects.


In [6]:
# Split into training and test set.
train_X, test_X, train_y, test_y = train_test_split(
    X, y, test_size=0.05, random_state=0)

# Create Dataset objects.
trainset = TensorDataset(torch.from_numpy(train_X), torch.from_numpy(train_y))
testset = TensorDataset(torch.from_numpy(test_X), torch.from_numpy(test_y))


We are now ready to build our model. From the perspective of training our model, it will look similar to the book sales prediction example, but we use a deeper model consisting of two LSTM layers (indicated by the argument num_layers to the nn.LSTM object). We want both LSTM layers to use a dropout value of 0.2. However, the nn.LSTM implementation does not apply dropout to the top layer, so we stack a separate Dropout layer on top of the LSTM object.

Just as in c9e1_rnn_book_sales we have to create a custom layer to retrieve the correct set of outputs from the nn.LSTM object. The return value from LSTM is slightly more complex in that it returns both the internal cell state as well as the output state of each layer so we now need yet another index to select only the output state. That is, the second index (0) indicates that we want the output of the layer instead of the cell state, and the third index (1) indicates that we want the output of the second LSTM layer.

We end with a fully connected layer with multiple neurons using a softmax function because we will be predicting probabilities for discrete entities (characters). We use categorical cross-entropy as our loss function, which is the recommended loss function for multicategory classification.

Finally, we train the model for 32 epochs with a mini-batch size of 256.


In [7]:
# Define model.
class LastTimestep(nn.Module):
    def forward(self, inputs):
        return inputs[1][0][1] # Return hidden state for last timestep.

model = nn.Sequential(
    nn.LSTM(encoding_width, 128, num_layers=2, dropout=0.2, batch_first=True),
    LastTimestep(),
    nn.Dropout(0.2), # Add this since PyTorch LSTM does not apply dropout to top layer.
    nn.Linear(128, encoding_width)
)

# Loss function and optimizer
optimizer = torch.optim.Adam(model.parameters())
loss_function = nn.CrossEntropyLoss()

# Train the model.
train_model(model, device, EPOCHS, BATCH_SIZE, trainset, testset,
            optimizer, loss_function, 'acc')


Epoch 1/32 loss: 2.8237 - acc: 0.2131 - val_loss: 2.4451 - val_acc: 0.2881
Epoch 2/32 loss: 2.3631 - acc: 0.3101 - val_loss: 2.2522 - val_acc: 0.3264
Epoch 3/32 loss: 2.2236 - acc: 0.3445 - val_loss: 2.1356 - val_acc: 0.3613
Epoch 4/32 loss: 2.1349 - acc: 0.3690 - val_loss: 2.0503 - val_acc: 0.3902
Epoch 5/32 loss: 2.0596 - acc: 0.3898 - val_loss: 1.9709 - val_acc: 0.4053
Epoch 6/32 loss: 1.9945 - acc: 0.4063 - val_loss: 1.9168 - val_acc: 0.4178
Epoch 7/32 loss: 1.9392 - acc: 0.4218 - val_loss: 1.8613 - val_acc: 0.4364
Epoch 8/32 loss: 1.8891 - acc: 0.4350 - val_loss: 1.8169 - val_acc: 0.4479
Epoch 9/32 loss: 1.8409 - acc: 0.4477 - val_loss: 1.7785 - val_acc: 0.4576
Epoch 10/32 loss: 1.7959 - acc: 0.4600 - val_loss: 1.7307 - val_acc: 0.4729
Epoch 11/32 loss: 1.7574 - acc: 0.4731 - val_loss: 1.6998 - val_acc: 0.4815
Epoch 12/32 loss: 1.7212 - acc: 0.4820 - val_loss: 1.6669 - val_acc: 0.4925
Epoch 13/32 loss: 1.6896 - acc: 0.4916 - val_loss: 1.6446 - val_acc: 0.4977
Epoch 14/32 loss: 1.6

[0.5788457103321033, 0.55859375]

The next step is to implement the beam search algorithm to predict text. Consult the section "Beam Search" in Chapter 11 for more details about beam search. In our implementation, each beam is represented by a tuple with three elements. The first element is the logarithm of the cumulative probability for the current sequence of characters. The second element is the string of characters. The third element is a one-hot encoded version of the string of characters. A reasonable question is why we store the logarithm of the cumulative probability instead of just the cumulative probability. Given that these probabilities are small, there is a risk that the limited precision of computer arithmetic results in underflow. This is addressed by instead computing the logarithm of the probability, in which case the multiplication is converted to an addition. For a small number of words, this is not necessary, but we do it anyway for good practice.

We start by creating a single beam with an initial sequence of characters ('the body ') and set the initial probability to 1.0. The one-hot encoded version of the string is created by the first loop. We add this beam to a list named beams.

This is followed by a nested loop that uses the trained model to do predictions according to the beam-search algorithm. We extract the one-hot encoding representation of each beam and create a NumPy array with multiple input examples. There is one input example per beam. During the first iteration, there is only a single input example. During the remaining iterations, there will be BEAM_SIZE number of examples.

We convert the input to a tensor, move to the GPU and feed to the model. We also need to explicitly apply softmax to the output because the softmax operation is not included in the model itself. This results in one softmax vector per beam. The softmax vector contains one probability per word in the vocabulary. For each beam, we create BEAM_SIZE new beams, each beam consisting of the words from the original beam concatenated with one more word. We choose the most probable words when creating the beams. The probability for each beam can be computed by multiplying the current probability of the beam by the probability for the added word.

Once we have created BEAM_SIZE beams for each existing beam, we sort the list of new beams according to their probabilities. We then discard all but the top BEAM_SIZE beams. This represents the pruning step. For the first iteration, this does not result in any pruning because we started with a single beam, and this beam resulted in just BEAM_SIZE beams. For all remaining iterations, we will end up with BEAM_SIZE * BEAM_SIZE beams and discard most of them.

The loop runs for a fixed number of iterations followed by printing out the generated predictions. See the section "Programming Example: Using LSTM for Text Autocompletion" in Chapter 12 for examples of generated predictions that an equivalent TensorFlow implementation generated.


In [8]:
# Create initial single beam represented by triplet
# (probability , string , one-hot encoded string).
letters = 'the body '
one_hots = []
for i, char in enumerate(letters):
    x = np.zeros(encoding_width)
    x[char_to_index[char]] = 1
    one_hots.append(x)
beams = [(np.log(1.0), letters, one_hots)]

# Predict NUM_LETTERS into the future.
for i in range(NUM_LETTERS):
    minibatch_list = []
    # Create minibatch from one-hot encodings, and predict.
    for triple in beams:
        minibatch_list.append(triple[2])
    minibatch = np.array(minibatch_list, dtype=np.float32)
    inputs = torch.from_numpy(minibatch)
    inputs = inputs.to(device)
    outputs = model(inputs)
    outputs = F.softmax(outputs, dim = 1)
    y_predict = outputs.cpu().detach().numpy()

    new_beams = []
    for j, softmax_vec in enumerate(y_predict):
        triple = beams[j]
        # Create BEAM_SIZE new beams from each existing beam.
        for k in range(BEAM_SIZE):
            char_index = np.argmax(softmax_vec)
            new_prob = triple[0] + np.log(softmax_vec[char_index])
            new_letters = triple[1] + index_to_char[char_index]
            x = np.zeros(encoding_width)
            x[char_index] = 1
            new_one_hots = triple[2].copy()
            new_one_hots.append(x)
            new_beams.append((new_prob, new_letters, new_one_hots))
            softmax_vec[char_index] = 0
    # Prune tree to only keep BEAM_SIZE most probable beams.
    new_beams.sort(key=lambda tup: tup[0], reverse=True)
    beams = new_beams[0:BEAM_SIZE]
for item in beams:
    print(item[1])
    

the body had been th
the body had been to
the body had been a 
the body of the coun
the body had been ha
the body of the cons
the body had been an
the body had been he
